In [2]:
using LinearAlgebra
using Optim
using HDF5

using QuantumOptics
basis = NLevelBasis(2)

include("../LiPoSID.jl")

using Statistics

┌ Warning: Package LinearSolve does not have KrylovKit in its dependencies:
│ - If you have LinearSolve checked out for development and have
│   added KrylovKit as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with LinearSolve
│ Loading KrylovKit into LinearSolve from project dependency, future warnings for LinearSolve are suppressed.
└ @ nothing nothing:984


In [3]:
using Dates

Fidelity for qubit can be written:

$F(\rho, \sigma) = \operatorname{tr}(\rho \sigma) + 2\sqrt{\det(\rho) \det(\sigma)}$

In [4]:
function infidelity_norm(ρ, σ)
    abs(1 - real(tr(ρ * σ)) - 2*sqrt(abs(det(ρ)*det(σ))))
end

infidelity_norm (generic function with 1 method)

In [5]:
function read_timeevolution(file_name, state, γ)
    h5open(file_name, "r") do file
        ρᵧ = read(file[state][string(γ)])
        t = ρᵧ["t"]
        ρ₀₀ = ρᵧ["p0"]; Re_ρ₀₁ = ρᵧ["s_re"];  Im_ρ₀₁ = ρᵧ["s_im"]
        ρ_series = []
        t_series = []

        for i in 1:length(t)
            ρᵢ= [ ρ₀₀[i]                      Re_ρ₀₁[i] + im * Im_ρ₀₁[i]
                  Re_ρ₀₁[i] - im * Im_ρ₀₁[i]  1 - ρ₀₀[i]                 ]
            push!(ρ_series, convert(Matrix{ComplexF64}, ρᵢ))
            push!(t_series, convert(Float64, t[i]))
        end
        return(t_series, ρ_series)
    end
end

# Define the Pauli matrices
σˣ = [0 1; 1 0]
σʸ = [0 im; -im 0]
σᶻ = [1 0; 0 -1]

# Define the basis elements
fᴷ₁ = σˣ / 2
fᴷ₂ = σʸ / 2
fᴷ₃ = σᶻ / 2

# Check orthogonality and normalization
@assert tr(fᴷ₁ * fᴷ₂) ≈ 0
@assert tr(fᴷ₁ * fᴷ₃) ≈ 0
@assert tr(fᴷ₂ * fᴷ₃) ≈ 0
@assert tr(fᴷ₁ * fᴷ₁) ≈ 1 / 2
@assert tr(fᴷ₂ * fᴷ₂) ≈ 1 / 2
@assert tr(fᴷ₃ * fᴷ₃) ≈ 1 / 2

fᴼᴺᴮ = [fᴷ₁, fᴷ₂, fᴷ₃]

# Function to calculate Dc
function Dc(ρ, t, H, C)
    U = (H * ρ - ρ * H) / im
    D = sum(C .* [2 * fᵢ * ρ * fⱼ' - ρ * fⱼ' * fᵢ - fⱼ' * fᵢ * ρ for fᵢ in fᴼᴺᴮ, fⱼ in fᴼᴺᴮ]) / 2
    return U + D
end

function construct_H(params)
    ϵ, h_Re, h_Im = params[1:3]
    return [
        ϵ               h_Re + im * h_Im
        h_Re - im * h_Im -ϵ
    ] / 2
end

function construct_C(params)
    m_Re = reshape(params[4:12], (3, 3))
    m_Im = reshape(params[13:21], (3, 3))
    M = m_Re + im * m_Im
    return M * M'
end


# Define the objective function for optimization
function kossak_obj(params, ρ, t)
    H = construct_H(params)
    C = construct_C(params)

    obj = 0.0
    for i in 3:length(ρ)
        ρ1 = ρ[i]
        ρ2 = ρ[i - 2] + (t[i] - t[i - 1]) * (Dc(ρ[i], t[i], H, C) + 4 * Dc(ρ[i - 1], t[i - 1], H, C) + Dc(ρ[i - 2], t[i - 2], H, C)) / 3
        obj += infidelity_norm(ρ1,ρ2)
    end
    return obj
end

# Define the objective function for optimization using RK4
function kossak_obj_old(params, ρ, t)
    H = construct_H(params)
    C = construct_C(params)

    # Use a let block to ensure optimal performance of the inner function
    let Dc_rk4 = (ρ, t, H, C, h) -> begin
        k1 = Dc(ρ, t, H, C)
        k2 = Dc(ρ + 0.5 * h * k1, t + 0.5 * h, H, C)
        k3 = Dc(ρ + 0.5 * h * k2, t + 0.5 * h, H, C)
        k4 = Dc(ρ + h * k3, t + h, H, C)
        return (k1 + 2*k2 + 2*k3 + k4) / 6
    end

        obj = 0.0
        for i in 2:length(ρ)
            ρ1 = ρ[i]
            h = t[i] - t[i - 1]
            ρ2 = ρ[i - 1] + h * Dc_rk4(ρ[i - 1], t[i - 1], H, C, h)
            obj += infidelity_norm(ρ1, ρ2)
        end
        return obj
    end
end


kossak_obj_old (generic function with 1 method)

In [6]:
# Define the objective function for optimization using RK4
function kossak_obj(params, ρ, t)
    H = construct_H(params)
    C = construct_C(params)

    # Use a let block to ensure optimal performance of the inner function
    let Dc_rk4 = (ρ, t, H, C, h) -> begin
        k1 = Dc(ρ, t, H, C)
        k2 = Dc(ρ + 0.5 * h * k1, t + 0.5 * h, H, C)
        k3 = Dc(ρ + 0.5 * h * k2, t + 0.5 * h, H, C)
        k4 = Dc(ρ + h * k3, t + h, H, C)
        return (k1 + 2*k2 + 2*k3 + k4) / 6
    end

        obj = 0.0
        for i in 2:length(ρ)
            ρ1 = ρ[i]
            h = t[i] - t[i - 1]
            ρ2 = ρ[i - 1] + h * Dc_rk4(ρ[i - 1], t[i - 1], H, C, h)
            obj += infidelity_norm(ρ1, ρ2)
        end
        return obj
    end
end

kossak_obj (generic function with 1 method)

In [7]:
# Define initial parameters
initial_params = [0.0, 0.0, 0.0,  # H parameters: ϵ, h_Re, h_Im
                  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  # m_Re parameters
                  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]; # m_Im parameters

initial_params .+= 1;

In [8]:
# Define the objective function wrapper
function objective(params)

    objGEXY = kossak_obj(params, ρᵍ, tᵍ) + kossak_obj(params, ρᵉ, tᵉ) + kossak_obj(params, ρˣ, tˣ) + kossak_obj(params, ρʸ, tʸ)

    return objGEXY
end

objective (generic function with 1 method)

In [9]:
# Define the density matrix evolution and time points

file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"

#γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]


γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

tests_data_file_name = "KOSSAK_UNCONSTR_RK_BFGS1_trn4_tst20_"*date_and_time_string * ".h5"

tests_dir = ""

ρᵍ₀ = [ 1 0.
        0 0 ]    # state to measure initial distance from

dodeca_10_states = ["D"*string(n) for n=1:10];

basis_states = ["B"*string(n) for n=1:4];

train_states = basis_states 
test_states = dodeca_10_states;


for γᵢ in γ

    println("γ =  "*γᵢ)

    γᶠ = parse(ComplexF64, γᵢ)

    tᵍ, ρᵍ = read_timeevolution(file_name, "B1", γᵢ)
    tᵉ, ρᵉ = read_timeevolution(file_name, "B2", γᵢ)
    tˣ, ρˣ = read_timeevolution(file_name, "B3", γᵢ)
    tʸ, ρʸ = read_timeevolution(file_name, "B4", γᵢ)

    function objective(params)

        objGEXY = kossak_obj(params, ρᵍ, tᵍ) + kossak_obj(params, ρᵉ, tᵉ) + kossak_obj(params, ρˣ, tˣ) + kossak_obj(params, ρʸ, tʸ)
    
        return objGEXY
    end

    # Perform the optimization
    result = optimize(objective, initial_params, 
    method = BFGS(),
    g_tol = 1e-12, #show_trace = true;
    autodiff = :forward)

    # Extract optimized parameters
    optimized_params = result.minimizer

    # Substitute the optimized parameters back into H and C
    optimized_H = construct_H(optimized_params)
    optimized_C = construct_C(optimized_params)

    Hˢⁱᵈ = convert.(ComplexF64,optimized_H)
    Cˢⁱᵈ = convert.(ComplexF64,optimized_C)

    effective_Lindblad = LiPoSID.get_lindblad_operators(Cˢⁱᵈ, fᴼᴺᴮ)

    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, "gamma_"*γᵢ) # create coupling group
        γ_group["H"] = convert.(ComplexF64, Hˢⁱᵈ)
        γ_group["C"] = convert.(ComplexF64, Cˢⁱᵈ)      
    end

    FminStates = []
    FmedianStates = []
    FmeanStates = []

    for state in test_states # loop over initial states
        
        print(state*" ")

        start_time = time()

        tₛ, ρₛ = read_timeevolution(file_name, state, γᵢ)
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
        #bᵗˢᵗ = LiPoSID.bloch(ρₛ)
        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
        tᵗˢᵗ = convert.(Float64, tₛ)

        #Simulated LME 
        #tˢⁱᵐ, ρˢⁱᵐ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis, Hˢⁱᵈ), [Jˢⁱᵐ])
        #bˢⁱᵐ = LiPoSID.bloch([ρᵢ.data for ρᵢ in ρˢⁱᵐ])

        ρₒ = DenseOperator(basis,ρₛ[1])
        dt = tᵗˢᵗ[2] - tᵗˢᵗ[1]
        tᵉⁿᵈ = tᵗˢᵗ[end]

        #print("effective_Lindblad_ops for Kossakowski")

        
        effective_Lindblad_ops = [DenseOperator(basis,j) for j in effective_Lindblad]

        #print("Simulating Kossakowski")

        tout, ρ_t_kossak = timeevolution.master(tᵗˢᵗ, ρₒ, DenseOperator(basis, Hˢⁱᵈ), effective_Lindblad_ops)
        ρˢⁱᵈ  = [ρₜ.data for ρₜ in ρ_t_kossak]

        #print("Calculating Fidelity")

        #F = LiPoSID.fidelity_series(basis, [ρₜ.data for ρₜ in ρˢⁱᵐ], ρˢⁱᵈ)
        F = LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, "gamma_"*γᵢ) # open coupling group
            init_state_group = create_group(γ_group, state) # create initial state group
            init_state_group["Fidelity"] = convert.(Float64, F)
        end
        
        FminState = minimum(F)
        FmedianState = median(F)
        FmeanState = mean(F)
        
        push!(FminStates, FminState)
        push!(FmedianStates, FmedianState)
        push!(FmeanStates, FmeanState)

    end

    # Calculate the mean
    #F_mean_value = mean(FmeanStates)

    # Calculate the median
    F_median_value = median(FmedianStates)

    # Calculate the min
    F_min_value = minimum(FminStates)

    println()
    println("Mimimal fidelity for "*γᵢ*": ", F_min_value)
    println("Median fidelity for "*γᵢ*": ", F_median_value)

end

println(tests_data_file_name)


γ =  0.079477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 0.079477: 0.9982887897200202
Median fidelity for 0.079477: 0.9991876853741517
γ =  0.25133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 0.25133: 0.9995042122174707
Median fidelity for 0.25133: 0.9999023831177267
γ =  0.79477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 0.79477: 0.9997340663367252
Median fidelity for 0.79477: 0.9998793122803562
γ =  2.5133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 2.5133: 0.9995033596598047
Median fidelity for 2.5133: 0.9998897305959866
γ =  7.9477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 7.9477: 0.9980412181455159
Median fidelity for 7.9477: 0.999887763497902
γ =  25.133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 25.133: 0.9399541620218295
Median fidelity for 25.133: 0.983102591090426
γ =  79.477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 79.477: 0.9703724002179632
Median fidelity for 79.477: 0.9914532634888369
γ =  251.33
D1 D2

γ =  0.079477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 0.079477: 0.9988243876971387
Median fidelity for 0.079477: 0.9995360257742397

γ =  0.25133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 0.25133: 0.9995188642141974
Median fidelity for 0.25133: 0.9999049135214302

γ =  0.79477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 0.79477: 0.9997383219721908
Median fidelity for 0.79477: 0.999882164228428

γ =  2.5133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 2.5133: 0.9995045127096831
Median fidelity for 2.5133: 0.9998910938936307

γ =  7.9477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 7.9477: 0.9980440474708095
Median fidelity for 7.9477: 0.9998879917704082

γ =  25.133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 25.133: 0.9608317871379254
Median fidelity for 25.133: 0.9903273356510316

γ =  79.477
...

D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 251.33: 0.9842271076206535
Median fidelity for 251.33: 0.9953059964935596
KOSSAK_UNCONSTR_BFGS1_trn4_tst20_2024-Jul-30_at_08-26.h5
Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...

D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 

Mimimal fidelity for 0.25133: 0.9995188642141652

Median fidelity for 0.25133: 0.9999049135214064